<a href="https://colab.research.google.com/github/wlswjdals/project_braille01/blob/main/arduino_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#include "braille.h"
#include <SoftwareSerial.h>

int dataPin = 2; // DATA 핀번호
int latchPin = 3; // LATCH 핀번호
int clockPin = 4; // CLOCK 핀번호
int no_module = 6; // 점자 출력기 연결 개수

braille bra(dataPin, latchPin, clockPin, no_module);

#define SIZE 150
char string_buffer[SIZE]; // 수신 받은 문자열
char string_buffer_serial[SIZE][4]; // 수신 받은 문자열을 글자 단위로 분리하여 배열에 저장
int str_char_count = 0; // 전체 문자개수

SoftwareSerial bluetooth(10, 11); // RX, TX 핀을 10번, 11번으로 설정

byte hangul_cho[19] = {
    0b00010000, // ㄱ
    0b00010000, // ㄲ
    0b00110000, // ㄴ
    0b00011000, // ㄷ
    0b00011000, // ㄸ
    0b00000100, // ㄹ
    0b00100100, // ㅁ
    0b00010100, // ㅂ
    0b00010100, // ㅃ
    0b00000001, // ㅅ
    0b00000001, // ㅆ
    0b00111100, // ㅇ
    0b00010001, // ㅈ
    0b00010001, // ㅉ
    0b00000101, // ㅊ
    0b00111000, // ㅋ
    0b00101100, // ㅌ
    0b00110100, // ㅍ
    0b00011100  // ㅎ
};

byte hangul2_cho[19] = {
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18
};

byte hangul_jung[21] = {
    0b00101001, // ㅏ
    0b00101110, // ㅐ
    0b00010110, // ㅑ
    0b00010110, // ㅒ
    0b00011010, // ㅓ
    0b00110110, // ㅔ
    0b00100101, // ㅕ
    0b00010010, // ㅖ
    0b00100011, // ㅗ
    0b00101011, // ㅘ
    0b00101011, // ㅙ
    0b00110111, // ㅚ
    0b00010011, // ㅛ
    0b00110010, // ㅜ
    0b00111010, // ㅝ
    0b00111010, // ㅞ
    0b00110010, // ㅟ
    0b00110001, // ㅠ
    0b00011001, // ㅡ
    0b00011101, // ㅢ
    0b00100110  // ㅣ
};

byte hangul2_jung[21] = {
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20
};

byte hangul_jong[28] = {
    0b00000000, // 없음
    0b00100000, // ㄱ
    0b00100000, // ㄲ
    0b00100000, // ㄳ
    0b00001100, // ㄴ
    0b00001100, // ㄵ
    0b00001100, // ㄶ
    0b00000110, // ㄷ
    0b00001000, // ㄹ
    0b00001000, // ㄺ
    0b00001000, // ㄻ
    0b00001000, // ㄼ
    0b00001000, // ㄽ
    0b00001000, // ㄾ
    0b00001000, // ㄿ
    0b00001000, // ㅀ
    0b00001001, // ㅁ
    0b00101000, // ㅂ
    0b00101000, // ㅄ
    0b00000010, // ㅅ
    0b00000010, // ㅆ
    0b00001111, // ㅇ
    0b00100010, // ㅈ
    0b00001010, // ㅊ
    0b00001110, // ㅋ
    0b00001011, // ㅌ
    0b00001101, // ㅍ
    0b00000111  // ㅎ
};

byte hangul2_jong[28] = {
    0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27
};

byte ascii_data[127] = {
    // ASCII 점자 데이터 정의
    // 각 문자의 데이터는 생략함
};

#define nextPin 6
#define prevPin 7

void setup() {
    Serial.begin(9600);
    bluetooth.begin(9600);
    bra.begin();
    delay(1000);
    bra.all_off();
    bra.refresh();
    pinMode(nextPin, INPUT);
    pinMode(prevPin, INPUT);
    Serial.println("HC-06 연결됨. 데이터를 수신하면 출력합니다.");
}

void loop() {
    if (bluetooth.available()) {
        String str = bluetooth.readStringUntil('\n');
        str.replace("\r", "");
        strcpy(string_buffer, str.c_str());
        Serial.println(string_buffer);
        processInput();
    }
}

void processInput() {
    int ind = 0;
    int len = strlen(string_buffer);
    int index = 0;

    while (ind < len) {
        int bytes = get_char_byte(string_buffer + ind);
        if (bytes == 1) {
            string_buffer_serial[index][0] = *(string_buffer + ind);
            memset(&string_buffer_serial[index][1], 0, 3);
            index++;
        } else if (bytes == 3) {
            memcpy(&string_buffer_serial[index][0], string_buffer + ind, 3);
            string_buffer_serial[index][3] = 0;
            index++;
        }
        ind += bytes;
    }
    str_char_count = index;
    displayBraille();
}

void displayBraille() {
    int i = -2;
    while (i < str_char_count) {
        if (digitalRead(nextPin) == HIGH) {
            delay(1000);
            handleNext(i);
        }
        if (digitalRead(prevPin) == HIGH) {
            delay(1000);
            handlePrev(i);
        }
        delay(2000);
    }
    Serial.println("작업 종료");
}

void handleNext(int &i) {
    if (string_buffer_serial[i][1] == 0) {
        handleAscii(i);
    } else {
        handleHangul(i);
    }
}

void handlePrev(int &i) {
    i -= 4;
    if (string_buffer_serial[i][1] == 0) {
        handleAscii(i);
    } else {
        handleHangul(i);
    }
}

void handleAscii(int &i) {
    int code1 = string_buffer_serial[i][0];
    int code4 = string_buffer_serial[i + 1][0];
    ascii_braille(code1, 0, 0, code4, 0, 0);
    delay(1000);
    i++;
}

void handleHangul(int &i) {
    unsigned int cho1, jung1, jong1;
    unsigned int cho2, jung2, jong2;
    split_han_cho_jung_jong(string_buffer_serial[i][0], string_buffer_serial[i][1], string_buffer_serial[i][2], cho1, jung1, jong1);
    i++;
    split_han_cho_jung_jong(string_buffer_serial[i][0], string_buffer_serial[i][1], string_buffer_serial[i][2], cho2, jung2, jong2);
    han_braille(cho1, jung1, jong1, cho2, jung2, jong2);
    delay(1000);
}

unsigned char get_char_byte(char *pos) {
    char val = *pos;
    if ((val & 0b10000000) == 0) {
        return 1;
    } else if ((val & 0b11100000) == 0b11000000) {
        return 2;
    } else if ((val & 0b11110000) == 0b11100000) {
        return 3;
    } else if ((val & 0b11111000) == 0b11110000) {
        return 4;
    } else if ((val & 0b11111100) == 0b11111000) {
        return 5;
    } else {
        return 6;
    }
}

void ascii_braille(int code1, int code2, int code3, int code4, int code5, int code6) {
    bra.all_off();
    for (int i = 0; i < 6; i++) {
        int on_off = ascii_data[code1] >> (5 - i) & 0b00000001;
        on_off ? bra.on(0, i) : bra.off(0, i);
    }
    for (int i = 0; i < 6; i++) {
        int on_off = ascii_data[code4] >> (5 - i) & 0b00000001;
        on_off ? bra.on(3, i) : bra.off(3, i);
    }
    bra.refresh();
}

void split_han_cho_jung_jong(char byte1, char byte2, char byte3, unsigned int &cho, unsigned int &jung, unsigned int &jong) {
    unsigned int utf16 = (byte1 & 0b00001111) << 12 | (byte2 & 0b00111111) << 6 | (byte3 & 0b00111111);
    unsigned int val = utf16 - 0xac00;
    unsigned char _jong = val % 28;
    unsigned char _jung = (val % (28 * 21)) / 28;
    unsigned char _cho = val / (28 * 21);

    cho = getHangulIndex(_cho, hangul2_cho, 19);
    jung = getHangulIndex(_jung, hangul2_jung, 21);
    jong = getHangulIndex(_jong, hangul2_jong, 28);
}

int getHangulIndex(unsigned char val, byte *array, int size) {
    for (int i = 0; i < size; i++) {
        if (val == array[i]) {
            return i;
        }
    }
    return 0;
}

void han_braille(int index1, int index2, int index3, int index4, int index5, int index6) {
    bra.all_off();
    setBraille(index1, 0, hangul_cho);
    setBraille(index2, 1, hangul_jung);
    setBraille(index3, 2, hangul_jong);
    setBraille(index4, 3, hangul_cho);
    setBraille(index5, 4, hangul_jung);
    setBraille(index6, 5, hangul_jong);
    bra.refresh();
}

void setBraille(int index, int module, byte *data) {
    for (int i = 0; i < 6; i++) {
        int on_off = data[index] >> (5 - i) & 0b00000001;
        on_off ? bra.on(module, i) : bra.off(module, i);
    }
}
